<a href="https://colab.research.google.com/github/MHoseinHoushmand/Clustering_by_SLFA/blob/main/Clustering_by_SLFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pdb
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
import operator
import random

In [60]:
categories = [  #Select 4 categories from fetch_20newsgroups dataset
    "alt.atheism",
    "comp.graphics",
    "sci.space",
    "rec.sport.hockey",
]

dataset = fetch_20newsgroups( #Preprocessing before using dataset
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)
labels = dataset.target[0:200]
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"{len(dataset.data[0:200])} documents - {true_k} categories")
print(labels)

200 documents - 4 categories
[3 2 0 0 1 1 2 3 2 1 3 1 3 0 0 2 1 0 3 1 0 2 1 0 2 2 1 1 3 3 0 1 1 2 3 3 2
 1 1 1 2 1 2 3 2 0 0 2 2 1 3 0 1 2 2 3 1 0 1 2 0 1 3 1 3 2 0 3 1 1 0 3 3 0
 1 0 2 0 2 1 2 2 0 2 3 2 1 2 0 2 0 1 1 3 3 2 0 2 0 3 1 0 0 1 2 3 3 0 3 3 2
 1 0 0 1 3 2 2 1 2 2 0 2 3 2 1 0 1 2 3 2 2 2 3 3 0 3 3 0 0 2 0 1 3 1 2 1 1
 3 2 3 3 0 1 3 3 2 2 2 3 2 0 2 0 1 3 3 1 0 2 2 2 2 3 2 3 1 1 1 1 2 3 3 1 2
 1 1 2 3 1 0 2 2 2 0 2 3 3 2 1]


In [69]:
# Vectorize all document as their term frequency(tfidf score)
def docs_as_tfidf(docs):
  vectorizer = TfidfVectorizer(
     max_df=0.5, #Removing terms that are used in more than 50% of articles
     min_df=5,   #Removing terms that are not used in less than 10 of articles
     stop_words="english",
     #  max_features=1000,
  )
  docs_vector = vectorizer.fit_transform(docs)
  return docs_vector.toarray()

In [47]:
population_size = 400 # Frogs number
memplex_num = 20 #define as m
memplex_size = 20 #define as n
max_iteration = 100 #Total Iteration
memplex_iteration = 10 #Iteration As local search
cluster_size = 4
docs = dataset.data[0:200]
docs_vector = docs_as_tfidf(docs)
print(list(docs_vector[0]))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.49823118076011713, 0.4354036764564651, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4504298033955598, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4426956398297136, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.40413227009900016, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
def cosin_sim(a,b):
   return cosine_similarity([a], [b])[0][0]

In [12]:
#Calculate sum of squared error(SSE) as similarity of each documents with the cluster mean in document
def SSE(cluster,doc_mean):
  size = len(cluster)
  sse=0
  for doc in cluster:
    sse += cosin_sim(doc,doc_mean)**2
  sse = sse/size
  return sse

In [13]:
#Calculate similarity between clusters
def BC(doc_means):
   BC=0
   size = len(doc_means)
   for i in range(size):
      for j in range(i+1,size):
          BC += cosin_sim(doc_means[i],doc_means[j])**2
   return BC

In [14]:
#Calculate similarity within clusters
def WC(clusters):
    WC = 0
    for cluster in clusters:
        doc_mean = np.average(cluster, axis=0)
        WC += SSE(cluster,doc_mean)
    return WC

In [61]:
def build_clusters(answer,docs_vector,cluster_size):
   clusters = []
   for i in range(cluster_size):
       clusters.append([])
   for j in range(len(answer)):
       if -1 < answer[j]:
        clusters[answer[j]].append(docs_vector[j])
   return clusters

In [33]:
def fitness(answer,docs_vector,size):
   doc_means = []
 #  pdb.runcall(build_clusters,answer,docs_vector,clusters_size)
   clusters = build_clusters(answer,docs_vector,size)
   for i in range(size):
       doc_means.append(
          np.average(clusters[i], axis=0)
       )
   wc = WC(clusters)
   bc = BC(doc_means)
   fitness = wc/bc
   return fitness

In [75]:
def cross_over(answer_a,answer_b):
    frog_size = len(answer_a)
    points = sorted(np.random.choice(np.arange(0,frog_size), size=2, replace=False))
    child1 = answer_a[:points[0]] + answer_b[points[0]:points[1]] + answer_a[points[1]:]
    child2 = answer_b[:points[0]] + answer_a[points[0]:points[1]] + answer_b[points[1]:]
 #   pdb.set_trace()
    fitness1 =  fitness(child1 ,docs_vector,cluster_size)
    fitness2 = fitness(child2 ,docs_vector,cluster_size)
    if fitness1 > fitness2:
       return child1 , fitness1
    else :
       return child2 , fitness2

In [20]:
def best_and_worst(answers):
     best =  max(answers, key=answers.get)
     worst = min(answers, key=answers.get)
     return tuple(best) , tuple(worst)


In [21]:
def global_best(memplexes):
     local_bests = {}
     for memplex in memplexes:
         local_best =  max(memplex, key=memplex.get)
         local_bests[local_best]= memplex[local_best]
     global_best = max(local_bests, key=local_bests.get)
     return global_best, local_bests[global_best]


In [22]:
def keys_to_remove(keys , dict):
   for k in keys:
      if k in dict:
          dict.pop(k)
   return dict

In [23]:
def mutation(global_best,clusters_size):
    new_ans = list(global_best)
    size = int(len(global_best)/4)
    indexes = np.random.choice(np.arange(0,len(global_best)), size=size, replace=False)
    values= [random.randint(0, 3) for _ in range(size)]
    for i in range(size):
      new_ans[indexes[i]] = values[i]
    return tuple(new_ans)

In [24]:
def Create_memplexes(population,memplex_num):
     population = dict( sorted(population.items(), key=operator.itemgetter(1), reverse=True))
     memplexes = []
     keys = list(population.keys())
     population_size = len(population)
     for i in range(memplex_num):
         memplexes.append({})
     for i in range(population_size):
         memplexes[i % memplex_num][keys[i]] = population[keys[i]]
     return memplexes

In [25]:
def create_submemplex(memplex,memplex_size, submemplex_size):
    sub_memplex = {}
    prob_list = []
    keys = []
    for i in range(memplex_size):
       for j in range(2*(memplex_size-i)):
          prob_list.append(i)
    k=0
    while(k!=submemplex_size):
       index = random.choice(prob_list)
       key = list(memplex.keys())[index]
       if key not in keys:
           sub_memplex[key] = memplex[key]
           keys.append(key)
           k+=1
    return sub_memplex

In [26]:
def shufeling(memplexes):
    output = {}
    for memplex in memplexes:
        output.update(memplex)
    return output

In [98]:
import operator
def frog_leaping_search(docs_vector):
             answers=np.random.randint(0, cluster_size, size=(population_size , len(docs)))
             population = {}
             i=0
             for answer in answers:
              #  pdb.runcall(fitness,answer,docs_vector,cluster_size)
                i+=1
                population[tuple(answer)] = fitness(answer,docs_vector,cluster_size)
                print(i,population[tuple(answer)])

             for i in range(max_iteration):
            #    pdb.runcall(Create_memplexes,population, memplex_num)
                memplexes = Create_memplexes(population, memplex_num)
                population.clear()
                pdb.set_trace()
                for j in range(memplex_num):
                    print(i,j,len(memplexes[j]))
                    #pdb.runcall(create_submemplex,memplexes[j],memplex_size, 5)
                    sub_memplex = create_submemplex(memplexes[j],memplex_size, 5)
                    memplexes[j] =  keys_to_remove(sub_memplex.keys(),memplexes[j])
                    for k in range(memplex_iteration):
                       # pdb.runcall(best_and_worst,sub_memplex)

                   #     for m in sub_memplex:\n",
                    #        print(list(m))
                        ans_best, ans_worst = best_and_worst(sub_memplex)
                        ans_out , fitness_out = cross_over(ans_best,ans_worst)
                        #sec B,
                        ###############################################
                      #  if len(sub_memplex)< 5:
                       #        print("errrrrrrrrrrrorrrrrrrrrrrrrB")
                        #       pdb.set_trace()
                      ###############################################
                      #pdb.runcall(best_and_worst,sub_memplex)
                       # print("############")


                        if (sub_memplex[ans_worst]<fitness_out):
                            del sub_memplex[ans_worst]
                            sub_memplex[ans_out] = fitness_out
                            #sec C
                           ###############################################
                       #     if len(sub_memplex)< 5:
                        #              print("errrrrrrrrrrrorrrrrrrrrrrrrC")
                         #             pdb.set_trace()
                          ###############################################
                        else:
                           # pdb.runcall(global_best,memplexes)
                            g_best, g_value = global_best(memplexes)
                            ans_out , fitness_out = cross_over(g_best,ans_worst)
                            #sec D
                            ###############################################
                     #       if len(sub_memplex)< 5:
                      #            print("errrrrrrrrrrrorrrrrrrrrrrrrD")
                       #           pdb.set_trace()
                            ###############################################
                            if (sub_memplex[ans_worst] < fitness_out):
                                del sub_memplex[ans_worst]
                                sub_memplex[ans_out] = fitness_out
                                #sec E
                                ###############################################
                    #            if len(sub_memplex)< 5:
                     #                 print("errrrrrrrrrrrorrrrrrrrrrrrrE")
                      #                pdb.set_trace()
                            ###############################################
                            else:
                           #     print(\"#########\")
                            #    for m in sub_memplex:
                             #        print(list(m))
                                del sub_memplex[ans_worst]
                             #   pdb.runcall(mutation,g_best,cluster_size)
                                ans_out = mutation(g_best,cluster_size)
                                fitness_out = fitness(ans_out,docs_vector,cluster_size)
                                sub_memplex[ans_out] = fitness_out
                                #sec F
                                ###############################################
                       #         if len(sub_memplex)< 5:
                        #              print("errrrrrrrrrrrorrrrrrrrrrrrrF")
                         #             pdb.set_trace()
                               ###############################################
                #     pdb.runcall(join_dicts,memplexes[j],sub_memplex)
                    memplexes[j].update(sub_memplex)
                    print(i,j,len(memplexes[j]))
                g_best, g_value = global_best(memplexes)
               # pdb.runcall(show_result,g_best)
                print(g_best)
                print(g_value)
                pdb.set_trace()
                population = shufeling(memplexes)
             return g_best, g_value, population

In [99]:
ng_best, g_value,population = frog_leaping_search(docs_vector)
true = 0
size = len(dataset.data)
print(labels)
print(list(g_best))

1 0.10394242787080316
2 0.1079140170005778
3 0.09750293108627162
4 0.09693019756910713
5 0.0956036871459985
6 0.10426686822742808
7 0.10666448215939038
8 0.09098853971549153
9 0.09558023809030658
10 0.10273482923706362
11 0.10542830765580224
12 0.10055754060627092
13 0.11533980430080373
14 0.09421161991815094
15 0.107578188363341
16 0.09539347886988393
17 0.10641907753418935
18 0.10799796694426814
19 0.0968061183127273
20 0.1029918194588913
21 0.11475462281646014
22 0.10746183432752007
23 0.09660033751793684
24 0.09494923879912633
25 0.0976726502512122
26 0.09287942829647264
27 0.10426390512769475
28 0.09843853276345084
29 0.09515183919215252
30 0.10031677268481347
31 0.10702351079375991
32 0.09841169090133296
33 0.10068834800176385
34 0.10338512636329555
35 0.09700549701802968
36 0.11066951416964962
37 0.09674137677569321
38 0.10068295063979857
39 0.09483693463229405
40 0.11334457192799105
41 0.10956848753738878
42 0.10655324985713041
43 0.09949081396429849
44 0.107266004499867
45 0.0

KeyboardInterrupt: ignored